## Содержание

[Библиотеки](#libraries)

[Данные](#data)

[1. Общая Triplet Loss модель](#main_tl_model)

[2. Построение/загрузка базы признаковых описаний](#feature_base)

[3. Метрические модели классификации](#metric_models)
* [3.1. Инициализация](#initialization)
* [3.2. Тестирование моделей](#models_test)

<a id='libraries'></a>
## Библиотеки

In [1]:
import os

import dataset.custom_datasets as cust_ds
import dataset.default_augmentations as def_augs
from models.triplet_loss import TripletLossModel, TripletLossFeaturesModel

from tools.vectorizer import Vectorizer
from tools.feature_base import FeaturesBase
import models.metric_tl_classifier as metr_tl_cl
from model_test.metric_model_test import test_classification

<a id='data'></a>
## Данные

In [2]:
datasets_path = 'G:\\diplom\\project\\triplet_loss_project_2_06.04\\data'
train_ds_path = os.path.join(datasets_path, 'sampled_train_marks_models')
test_ds_path = os.path.join(datasets_path, 'sampled_test')

In [3]:
def models_indx_f(x):
    return (x[0], x[1])

In [4]:
classif_ds_train = cust_ds.TLClassifyDataset(
    train_ds_path, models_indx_f, crop=False,
    transform=def_augs.clTL_train_augs_128
)

classif_ds_test = cust_ds.TLClassifyDataset(
    test_ds_path, models_indx_f, crop=False,
    transform=def_augs.clTL_test_augs_128
)

<a id='main_tl_model'></a>
## 1. Общая Triplet Loss модель

In [5]:
marks_features_len = 128
tl_mark_model = TripletLossModel(marks_features_len)

weights_marks_p = [
    'weights/2. triplet loss model - marks/3. 1154 epochs/conv2Dfeatures_best_var.pt',
    'weights/2. triplet loss model - marks/3. 1154 epochs/fully_connect_best_var.pt'
]

tl_mark_model.split_load(weights_marks_p)

In [6]:
models_features_len = 64
tl_models_model = TripletLossModel(models_features_len)

weights_models_p = [
    'weights/4. triplet loss model - models/2. 456 epochs/conv2Dfeatures_best_var.pt',
    'weights/4. triplet loss model - models/2. 456 epochs/fully_connect_best_var.pt'
]

tl_models_model.split_load(weights_models_p)

In [7]:
total_features_len = marks_features_len + models_features_len
cat_tl_model = TripletLossFeaturesModel(total_features_len,
                                        tl_mark_model, tl_models_model)

<a id='feature_base'></a>
## 2. Построение/загрузка базы признаковых описаний

In [8]:
features_base = FeaturesBase(total_features_len)

In [9]:
# features_base.fill_from_path(train_ds_path, cat_tl_model)

In [10]:
base_filepaths = [
    'csv/train/data_marks_train.csv',
    'csv/train/data_models_train.csv',
    'csv/train/data_all_train.csv'
]
# features_base.save_base(base_filepaths)
features_base.load_base(base_filepaths)

In [11]:
len(features_base.data_all['ID'])

20883

<a id='metric_models'></a>
## 3. Метрические модели классификации

<a id='initialization'></a>
### 3.1. Инициализация

Базовые метрические модели:

In [12]:
knn_marks_model = metr_tl_cl.MarkskNN(features_base)
knn_marks_model.set_knn()

knn_models_model = metr_tl_cl.ModelskNN(features_base)
knn_models_model.set_knn()

mah_marks_model = metr_tl_cl.MarksMah(features_base)
mah_marks_model.set_mah()

mah_models_model = metr_tl_cl.ModelsMah(features_base)
mah_models_model.set_mah()

Комбинации базовых моделей:

In [13]:
knn_knn_model = metr_tl_cl.MetricClassificationModel(knn_marks_model,
                                                     knn_models_model)

knn_mah_model = metr_tl_cl.MetricClassificationModel(knn_marks_model,
                                                     mah_models_model)

mah_knn_model = metr_tl_cl.MetricClassificationModel(mah_marks_model,
                                                     knn_models_model)

mah_mah_model = metr_tl_cl.MetricClassificationModel(mah_marks_model,
                                                     mah_models_model)

<a id='models_test'></a>
### 3.2. Тестирование моделей

In [14]:
res0 = test_classification(test_ds_path, cat_tl_model, knn_knn_model)
print('Результаты knn + knn:')
print(f'marks acc: {round(res0[0], 3)}')
print(f'models acc: {round(res0[1], 3)}')
print(f'marks f1: {round(res0[2], 3)}')
print(f'models f1: {round(res0[3], 3)}')

  0%|          | 0/5335 [00:00<?, ?it/s]

Результаты knn + knn:
marks acc: 0.648
models acc: 0.063
marks f1: 0.653
models f1: 0.049


In [15]:
res1 = test_classification(test_ds_path, cat_tl_model, knn_mah_model)
print('Результаты knn + mah:')
print(f'marks acc: {round(res1[0], 3)}')
print(f'models acc: {round(res1[1], 3)}')
print(f'marks f1: {round(res1[2], 3)}')
print(f'models f1: {round(res1[3], 3)}')

  0%|          | 0/5335 [00:00<?, ?it/s]

C:\Users\ski6a\Desktop\Junior ML Contest\src\models\metric_tl_classifier.py:34: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt(np.einsum('ij,ji->i', a, b))


Результаты knn + mah:
marks acc: 0.648
models acc: 0.035
marks f1: 0.653
models f1: 0.014


In [16]:
res2 = test_classification(test_ds_path, cat_tl_model, mah_knn_model)
print('Результаты mah + knn:')
print(f'marks acc: {round(res2[0], 3)}')
print(f'models acc: {round(res2[1], 3)}')
print(f'marks f1: {round(res2[2], 3)}')
print(f'models f1: {round(res2[3], 3)}')

  0%|          | 0/5335 [00:00<?, ?it/s]

Результаты mah + knn:
marks acc: 0.644
models acc: 0.064
marks f1: 0.657
models f1: 0.048


In [17]:
res3 = test_classification(test_ds_path, cat_tl_model, mah_mah_model)
print('Результаты mah + knn:')
print(f'marks acc: {round(res3[0], 3)}')
print(f'models acc: {round(res3[1], 3)}')
print(f'marks f1: {round(res3[2], 3)}')
print(f'models f1: {round(res3[3], 3)}')

  0%|          | 0/5335 [00:00<?, ?it/s]

C:\Users\ski6a\Desktop\Junior ML Contest\src\models\metric_tl_classifier.py:34: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt(np.einsum('ij,ji->i', a, b))


Результаты mah + knn:
marks acc: 0.644
models acc: 0.037
marks f1: 0.657
models f1: 0.013
